In [1]:
import random
import deap
import numpy as np

from deap import base
from deap import creator
from deap import tools

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
'''
適合度を最適化（最大化）
個体
'''
toolbox = base.Toolbox()
# Attribute generator(個体の作成)
toolbox.register("attr",random.randint,48,96)
toolbox.register("attr_rhythm",random.choice,0.25,0.5,1,2,4)
toolbox.register("attr_code",random.randint,70,76)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr, 64)#initRepeat　データを入れるコンテナの型、データを生成する関数、コンテナのサイズを引数ロする集団を発生させる関数
toolbox.register("rindividual", tools.initRepeat, creator.Individual, toolbox.attr_rhythm,10)
toolbox.register("cindividual", tools.initRepeat, creator.Individual, toolbox.attr_code,8)
toolbox.register("population1", tools.initRepeat, list, toolbox.individual)#メロディ
toolbox.register("population2", tools.initRepeat, list, toolbox.cindividual) #コード

In [2]:
#案１(botu)　音符の落差最小化

def evalOneMax(individual):
    score = 0
    for i in range(0,len(individual)-1):
        if (i >= len(individual)):
            break
        score -= abs(individual[i] - individual[i+1])
    score += sum(individual)
    return score,
'''
この部分を音楽理論に基づいた評価にしたい
方針１：
　制約
 　　１オクターブ以上の音の跳躍を許さない
   　キーにない音を含んだ場合減点
'''


'\nこの部分を音楽理論に基づいた評価にしたい\n方針１：\n\u3000制約\n \u3000\u3000１オクターブ以上の音の跳躍を許さない\n   \u3000キーにない音を含んだ場合減点\n'

In [3]:
#案2　調にない音を減点
def evalmel(individual):
    score = 0 
    tonic= 60
    for i in range(0,len(individual)-1):
        if (i >= len(individual)):
            break
        score -= abs(tonic-individual[i])** 2
        if(abs(individual[i]-individual[i+1])>12):
            score -=300
        if(individual[i] % 12 == 0 or 2 or 4 or 7 or 9):
            score += 300
        elif(individual[i] % 12 == 5 or 11):
            score += 100
        else:
            score *= 0
        
    return score,

In [4]:
def evalcode(cindividual):
    score = 0 
    sc = 0
    
    triad = {0:'T',1:'S',2:'T',3:'S',4:'D',5:'S',6:'D'}
    for i in range(0,len(cindividual)-1):
        if (i >= len(cindividual)):
            if(triad[(cindividual[i]) %7]  == 'T' ):
                score += 40
            break
        
        if(triad[(cindividual[i]) % 7] == triad[cindividual[i+1] % 7]):
            sc +=1
            if((cindividual[i]) % 7 == cindividual[i+1] % 7): #同じコードが連続していたら減点
                sc += 1
        if(triad[(cindividual[i]) % 7] == 'D' and triad[(cindividual[i+1]) % 7] == 'S'): #ドミナント→サブドミナントは減点
            score -= 100
        
    score -= (sc ** 2)
        
    return score,

In [5]:
toolbox.register("evaluate1", evalmel)#評価関数
toolbox.register("evaluate2", evalcode)
toolbox.register("mate", tools.cxTwoPoint)#交叉
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)#突然変異
toolbox.register("select", tools.selTournament,tournsize=3)#選択

In [6]:
#コード生成
code = list()
def code_generate(code):
    
    random.seed(40)
    # 初期の個体群を生成 N体作成する
    pop = toolbox.population2(n=100)
    
    CXPB, MUTPB, NGEN = 0.6, 0.05, 300 # 交差確率、突然変異確率、進化計算のループ回数
    '''
    この部分を、印象値入力にできないだろうか
    '''
    print("Start of evolution")

    # 初期の個体群の評価
    fitnesses = list(map(toolbox.evaluate2,pop))
    print(fitnesses)
    print('nyaan')
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    print("  Evaluated %i individuals" % len(pop))

    # 進化計算開始
    for g in range(NGEN):
        print("-- Generation %i --" % g)
        if (g % 5 ==0):
            print("続行しますか？")
            print ('終了する場合は”quit”と入力してください。')
            is_continue = input('>>>  ')
            if (is_continue == 'quit'):
                break
            else:
                continue
                
        # 次世代の個体群
        offspring = toolbox.select(pop, len(pop))
        # 個体群のクローンを生成
        offspring = list(map(toolbox.clone, offspring))

        # 選択した個体群に交差と突然変異を適応する
        # 偶数番目と奇数番目の個体を取り出して交差
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # 適合度が計算されていない個体を集めて適合度を計算
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate2, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print("  Evaluated %i individuals" % len(invalid_ind))

        # 次世代群をoffspringにする
        pop[:] = offspring
        
        # すべての個体の適合度を配列にする
        fits = [ind.fitness.values[0] for ind in pop]

        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)

    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    code[:] = best_ind[:]
    return code

In [7]:
#メロディ生成
melody = list()
def melody_generate(music):
    print ('作成したい曲の調の番号を入力してください。0:\'C\',1:\'C\#\',2:\'D\',3:\'D\#\',4:\'E\',5:\'F\',6:\'F\#\',7:\'G\',8:\'G\#\',9:\'A\',10:\'A\#\',11:\'H\'')
    tonic = input('>>>  ') 
    tonic = int(tonic) + 60
    print(tonic)
    
    random.seed(60)
    # 初期の個体群を生成 N体作成する
    pop = toolbox.population1(n=300)
    
    CXPB, MUTPB, NGEN = 0.6, 0.3, 300 # 交差確率、突然変異確率、進化計算のループ回数
    '''
    この部分を、印象値入力にできないだろうか
    '''
    print("Start of evolution")

    # 初期の個体群の評価
    fitnesses = list(map(toolbox.evaluate1,pop))
    print(fitnesses)
    print('nyaan')
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    print("  Evaluated %i individuals" % len(pop))

    # 進化計算開始
    for g in range(NGEN):
        print("-- Generation %i --" % g)
        if (g % 30 ==0):
            print("続行しますか？")
            print ('終了する場合は”quit”と入力してください。')
            is_continue = input('>>>  ')
            if (is_continue == 'quit'):
                break
            else:
                continue
                
        # 次世代の個体群
        offspring = toolbox.select(pop, len(pop))
        # 個体群のクローンを生成
        offspring = list(map(toolbox.clone, offspring))

        # 選択した個体群に交差と突然変異を適応する
        # 偶数番目と奇数番目の個体を取り出して交差
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # 適合度が計算されていない個体を集めて適合度を計算
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate1, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print("  Evaluated %i individuals" % len(invalid_ind))

        # 次世代群をoffspringにする
        pop[:] = offspring
        
        # すべての個体の適合度を配列にする
        fits = [ind.fitness.values[0] for ind in pop]

        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)

    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    music[:] = best_ind[:]
    return music

In [8]:
code_generate(code)

Start of evolution
[(-16,), (-36,), (-104,), (-104,), (-125,), (-9,), (-4,), (-100,), (-109,), (-16,), (-116,), (-101,), (-225,), (-204,), (-16,), (-164,), (-4,), (-104,), (-149,), (-300,), (-201,), (-64,), (-201,), (-116,), (-100,), (-36,), (-16,), (-116,), (-204,), (-9,), (-16,), (-116,), (-9,), (-116,), (-116,), (-1,), (-104,), (-4,), (-64,), (-104,), (-104,), (-209,), (-116,), (-4,), (-36,), (-36,), (-125,), (-9,), (-109,), (-125,), (-4,), (-136,), (-200,), (-16,), (-201,), (-116,), (-101,), (-201,), (-25,), (-149,), (-9,), (-9,), (-16,), (-209,), (-136,), (-116,), (-125,), (-104,), (-109,), (-181,), (-1,), (-104,), (0,), (-101,), (-16,), (-104,), (-16,), (-109,), (-25,), (-125,), (-104,), (-204,), (-204,), (-9,), (-109,), (-36,), (-209,), (-209,), (-125,), (-201,), (-116,), (-136,), (-9,), (-25,), (-9,), (-216,), (-4,), (-109,), (-9,), (-9,)]
nyaan
  Evaluated 100 individuals
-- Generation 0 --
続行しますか？
終了する場合は”quit”と入力してください。
>>>  quit
-- End of (successful) evolution --
Best indi

[74, 70, 75, 76, 70, 73, 72, 73]

In [9]:
melody_generate(melody)

作成したい曲の調の番号を入力してください。0:'C',1:'C\#',2:'D',3:'D\#',4:'E',5:'F',6:'F\#',7:'G',8:'G\#',9:'A',10:'A\#',11:'H'
>>>  0
60
Start of evolution
[(-9457,), (-15122,), (-13440,), (-17878,), (-11264,), (-17928,), (-12852,), (-14485,), (-6690,), (-8807,), (-16242,), (-11581,), (-8756,), (-13294,), (-15164,), (-14962,), (-15460,), (-17684,), (-9472,), (-19243,), (-12701,), (-9734,), (-10500,), (-14452,), (-16058,), (-16388,), (-13418,), (-13277,), (-16803,), (-11243,), (-10749,), (-12849,), (-13908,), (-9171,), (-10523,), (-14276,), (-14517,), (-13335,), (-7937,), (-2562,), (-15276,), (-12955,), (-12013,), (-15812,), (-14512,), (-19697,), (-10665,), (-11262,), (-5834,), (-12721,), (-11774,), (-13633,), (-21232,), (-16503,), (-12768,), (-10815,), (-9181,), (-8802,), (-13860,), (-13119,), (-4303,), (-18967,), (-16845,), (-13921,), (-13474,), (-15697,), (-13432,), (-9851,), (-9932,), (-16977,), (-8377,), (-14193,), (-11394,), (-10689,), (-10872,), (-12323,), (-12873,), (-9755,), (-7256,), (-13017,), (-1

[55,
 84,
 83,
 73,
 69,
 49,
 60,
 56,
 73,
 54,
 72,
 77,
 81,
 72,
 64,
 62,
 89,
 68,
 57,
 64,
 54,
 54,
 48,
 55,
 58,
 64,
 62,
 56,
 81,
 66,
 82,
 76,
 55,
 52,
 93,
 82,
 55,
 68,
 64,
 54,
 70,
 82,
 94,
 57,
 67,
 70,
 65,
 82,
 55,
 52,
 57,
 50,
 95,
 58,
 61,
 65,
 75,
 48,
 63,
 66,
 53,
 58,
 82,
 65]

In [10]:
import pretty_midi as pm
#ノート番号の配列を元にMIDI楽曲ファイルを作る


def makemidi(mlist,clist,bpm,mname):
    print ('作成したい曲の調の番号を入力してください。0:\'C\',1:\'C\#\',2:\'D\',3:\'D\#\',4:\'E\',5:\'F\',6:\'F\#\',7:\'G\',8:\'G\#\',9:\'A\',10:\'A\#\',11:\'H\'')
    tonic = input('>>>  ')
    tonic = int(tonic) + 60
    print(tonic)
    song = pm.PrettyMIDI()             #楽曲ファイルを生成
    piano = pm.Instrument(program=1)   #音色にピアノをセット
    s=0                                #時間の初期化
    delta = 60 / bpm
    
    harmony = list()
    
    tonedic = {0:0,1:2,2:4,3:5,4:7,5:9,6:11}
    
    for j in range(len(clist)):
        if(clist[j]%7 == 0 or 3 or 4):
            harmony.append([(tonedic[clist[j] % 7]+ tonic),(tonedic[clist[j] % 7]+ tonic + 4),(tonedic[clist[j] % 7]+ tonic + 7)])
        elif(clist[j]%7 == 1 or 2 or 5):
            harmony.append([(tonedic[clist[j] % 7]+ tonic),(tonedic[clist[j] % 7]+ tonic + 3),(tonedic[clist[j] % 7]+ tonic + 7)])
        else:
            harmony.append([(tonedic[clist[j] % 7]+ tonic),(tonedic[clist[j] % 7]+ tonic + 3),(tonedic[clist[j] % 7]+ tonic + 6)])
        print(harmony) 
        harmony.reverse()
        p1=harmony.pop()
        for k in range(3):
            note=pm.Note(velocity =48, pitch = p1.pop(), start =s, end = s + delta * 2) 
            piano.notes.append(note)
        s += delta*4
        print(s)
        print(delta)
        print(len(mlist))
        print(note)
        
    
    s=0
    
    for i in range(len(mlist)):
        print('u')
        if mlist[i] > 127:   #ノート番号が１２７を超えた場合休符にする
            mlist[i] = 0
        
        note=pm.Note(velocity =64, pitch = mlist[i], start =s, end = s + delta * 0.5) 
        s += delta*0.5
        print(s)
        print(delta)
        print(len(mlist))
        piano.notes.append(note)

    
    song.instruments.append(piano)

    song.write('{0}.mid'.format(mname))#song.midに書き出し
    print('Success to make {0}.mid !!\n'.format(mname))

In [11]:
makemidi(melody,code,120,'300gen')

作成したい曲の調の番号を入力してください。0:'C',1:'C\#',2:'D',3:'D\#',4:'E',5:'F',6:'F\#',7:'G',8:'G\#',9:'A',10:'A\#',11:'H'
>>>  0
60
[[67, 71, 74]]
2.0
0.5
64
Note(start=0.000000, end=1.000000, pitch=67, velocity=48)
[[60, 64, 67]]
4.0
0.5
64
Note(start=2.000000, end=3.000000, pitch=60, velocity=48)
[[69, 73, 76]]
6.0
0.5
64
Note(start=4.000000, end=5.000000, pitch=69, velocity=48)
[[71, 75, 78]]
8.0
0.5
64
Note(start=6.000000, end=7.000000, pitch=71, velocity=48)
[[60, 64, 67]]
10.0
0.5
64
Note(start=8.000000, end=9.000000, pitch=60, velocity=48)
[[65, 69, 72]]
12.0
0.5
64
Note(start=10.000000, end=11.000000, pitch=65, velocity=48)
[[64, 68, 71]]
14.0
0.5
64
Note(start=12.000000, end=13.000000, pitch=64, velocity=48)
[[65, 69, 72]]
16.0
0.5
64
Note(start=14.000000, end=15.000000, pitch=65, velocity=48)
u
0.25
0.5
64
u
0.5
0.5
64
u
0.75
0.5
64
u
1.0
0.5
64
u
1.25
0.5
64
u
1.5
0.5
64
u
1.75
0.5
64
u
2.0
0.5
64
u
2.25
0.5
64
u
2.5
0.5
64
u
2.75
0.5
64
u
3.0
0.5
64
u
3.25
0.5
64
u
3.5
0.5
64
u
3.75
0.5
64
